In [1]:
import os
from scipy import stats
import numpy as np
from scipy.io import loadmat
from scipy.spatial.distance import cdist
from sklearn import decomposition
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import matplotlib
import pickle
import glob
import pandas as pd
from warnings import warn
import seaborn as sns
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
# matplotlib.rc('font', **{'family': 'sans-serif', 'sans-serif': ['Arial']})


In [2]:
# -*- coding: utf-8 -*-
"""
Created on Thu May 12 16:59:04 2022

@author: lindseyb
"""


def get_regions(data, M, threshold, width, base_thresh):
    '''
    find all regions in the average trace where peaks of sufficient height and width occur
    '''
    upreg = np.where(data > (threshold*M)
                     )[0]  # all indices where data greater than .25*max
    baseline = np.mean(data[~upreg])
    regions = []
    if len(upreg) == 0:
        return regions
    last = upreg[0]
    i = 1
    currentreg = [last]
    while i < (len(upreg)-1):  # continue appending onto region while still upregulated
        curr = upreg[i]
        if curr == last+1:
            currentreg.append(curr)
        else:
            if len(currentreg) > width:  # only consider region upregulated if sufficiently long
                regions.append(currentreg)
            currentreg = [curr]
        last = curr
        i = i+1
    if len(currentreg) > width:
        if np.mean(data[currentreg]) > base_thresh*baseline:  # lines added for Ryan's method
            regions.append(currentreg)
    return regions


def divide_LR(alldata, leftchoices, rightchoices, pthresh, region_threshold, region_width, basethresh):
    '''

    Parameters
    ----------
    alldata : neural data (trials x neuron x position)
    leftchoices : trials in which the animal went left
    rightchoices : trials in which the animal went right

    Returns
    -------
    indices of left preferrring neurons, indices of right preferring neurons, 
    and indices of neurons with no significant difference in response between
    the two choices

    '''
    avgdata = np.mean(alldata, axis=0)  # neurons x position

    # transform data by substracting the minimum
    avgdata = avgdata - np.reshape(np.min(avgdata, axis=1), (-1, 1))

    left_neurons = []
    right_neurons = []
    split_neurons = []  # neurons are still task modulated but not significant choice selective
    nonmod_neurons = []  # neurons with no peaks

    maxfire = np.max(avgdata, axis=1)  # find the maximum firing rate
    # activity when the animal made a right choice
    rightfires = alldata[rightchoices, :, :]
    # activity when the animal made a left choice
    leftfires = alldata[leftchoices, :, :]
    for i, m in enumerate(maxfire):  # find peaks for each neuron
        upregions = get_regions(
            avgdata[i, :], m, region_threshold, region_width, base_thresh)
        left = False
        right = False
        for region in upregions:  # check if any regions are significantly different
            leftfiring = leftfires[:, i, region]
            leftactivity = np.mean(leftfiring, axis=1)
            rightfiring = rightfires[:, i, region]
            rightactivity = np.mean(rightfiring, axis=1)
            # ttest for difference in means
            tval, pval = stats.ttest_ind(leftactivity, rightactivity)
            if pval < 2*pthresh:  # mean activity unequal, one-sided
                if np.mean(leftactivity) > np.mean(rightactivity):
                    left = True
                else:
                    right = True
        if not (right and left):
            if right:  # only right upregulated regions
                right_neurons.append(i)
            elif left:  # only left upregulated regions
                left_neurons.append(i)
            else:  # greater activity depends on the region
                if len(upregions) > 0:
                    split_neurons.append(i)
                else:
                    nonmod_neurons.append(i)
        else:
            if len(upregions) > 0:
                # upregulated regions but never significantly different activity
                split_neurons.append(i)
            else:
                nonmod_neurons.append(i)
    return np.array(left_neurons), np.array(right_neurons), np.array(split_neurons), np.array(nonmod_neurons)


In [3]:
with open('test_data_acc_ind_492_0607.pickle', 'rb') as handle:
    data = pickle.load(handle)
print(data.keys())

dict_keys(['nCues_RminusL', 'currMaze', 'laserON', 'trialStart', 'trialEnd', 'keyFrames', 'time', 'cueOnset_L', 'cueOnset_R', 'choice', 'trialType', 'spikes', 'timeSqueezedFR'])


In [4]:
bin_sizes = [5, 25, 15, 7, 15]
alldata_lsr = [None, None]
rightis_lsr = [None, None]
leftis_lsr = [None, None]

# files = os.listdir()
# matfiles = [f for f in files if f.startswith('tet')]
bin_sizes = [5, 25, 15, 7, 15]
 # set parameters for Sue Ann's method
pthresh = .05
# percentage of max to be an upregulated region (Ryan uses .5)
region_threshold = .25
region_width = 3  # number of consecutive bins to be an upregulated region
# threshold introduced by Ryan (3) difference between down and upregulated regions
base_thresh = 0

n_bins = data['timeSqueezedFR'][0].shape[1]

# 66 different positions in Ryan's processed data
LCellLChoice = np.zeros((1, n_bins))
LCellRChoice = np.zeros((1, n_bins))

RCellLChoice = np.zeros((1, n_bins))
RCellRChoice = np.zeros((1, n_bins))

SCellLChoice = np.zeros((1, n_bins))
SCellRChoice = np.zeros((1, n_bins))

NCellLChoice = np.zeros((1, n_bins))
NCellRChoice = np.zeros((1, n_bins))

# trial_idx = [None for i in idx]  

    # list of np.ndarrays containing indices for which trials to use (only where it maze 10 or 11)

laser = 0
trial_idx = np.nonzero((data['currMaze'] > 7)*(data['laserON'] == laser))[0]
# data = loadmat(file)

    # create 3d array (trials x neurons x timepoints)
n_neurons = len(data['spikes'])

[n_trials, n_bins] = data['timeSqueezedFR'][0][trial_idx, :].shape

alldata = np.zeros((n_trials, n_neurons, n_bins))

# maintrials = data['Epoch']['Trial_Main_Maze'][0][0][0] - \
#     1  # subtract one for difference in matlab indexing
correcttrials = np.where(data['choice'][trial_idx] == data['trialType'][trial_idx], 1, 0)
print(correcttrials.size)
# print(f'ncorrect: {correcttrials.sum()}/{correcttrials.size}')

    # get data normalized by position
for i in range(n_neurons):
    alldata[:, i, :] = data['timeSqueezedFR'][i][trial_idx, :]

alldata[np.isnan(alldata)] = 0  # replace nan values with 0

leftchoices_correct = np.where((correcttrials==1)*(data['choice'][trial_idx]==0))[0]
# print(leftchoices_correct.size)
leftchoices_incorrect = np.where((correcttrials==0)*(data['choice'][trial_idx]==0))[0]
# print(leftchoices_incorrect.size)
rightchoices_correct = np.where((correcttrials==1)*(data['choice'][trial_idx]==1))[0]
# print(rightchoices_correct.size)
rightchoices_incorrect = np.where((correcttrials==0)*(data['choice'][trial_idx]==1))[0]
# print(rightchoices_incorrect.size)

lchoices = np.concatenate((leftchoices_correct, leftchoices_incorrect))
rchoices = np.concatenate((rightchoices_correct, rightchoices_incorrect))
# print(lchoices)
# print(alldata.shape)

avgdata = np.mean(alldata[:, :, :], axis=0)  # neurons x position

# divide each neuron as left pref, right pref, neither, or non-modulated
leftis, rightis, splitis, nonis = divide_LR(
    alldata, lchoices, rchoices, pthresh, region_threshold, region_width, base_thresh)

leftdata = alldata[lchoices, :, :]
rightdata = alldata[rchoices, :, :]

# get data for sequence plots grouped by animal's choice and neuron preference
try:
    leftcellsleftchoice = np.mean(leftdata[:, leftis, :], axis=0)
    LCellLChoice = np.vstack((LCellLChoice, leftcellsleftchoice))
except:
    print(leftis)
    warn('no left cells left choice')

try:
    leftcellsrightchoice = np.mean(rightdata[:, leftis, :], axis=0)
    LCellRChoice = np.vstack((LCellRChoice, leftcellsrightchoice))
except:
    print(leftis)
    warn('no left cells right choice')

try:
    rightcellsleftchoice = np.mean(leftdata[:, rightis, :], axis=0)
    RCellLChoice = np.vstack((RCellLChoice, rightcellsleftchoice))
except:
    print(rightis)
    warn('no right cells left choice')

try:
    rightcellsrightchoice = np.mean(rightdata[:, rightis, :], axis=0)
    RCellRChoice = np.vstack((RCellRChoice, rightcellsrightchoice))
except:
    print(rightis)
    warn('no right cells right choice')

try:
    splitcellsleftchoice = np.mean(leftdata[:, splitis, :], axis=0)
    SCellLChoice = np.vstack((SCellLChoice, splitcellsleftchoice))
except:
    warn('no split cells left choice')

try:
    splitcellsrightchoice = np.mean(rightdata[:, splitis, :], axis=0)
    SCellRChoice = np.vstack((SCellRChoice, splitcellsrightchoice))
except:
    warn('no split cells left choice')

if len(nonis) > 0:
    nonmodcellsleftchoice = np.mean(leftdata[:, nonis, :], axis=0)
    nonmodcellsrightchoice = np.mean(rightdata[:, nonis, :], axis=0)
    NCellLChoice = np.vstack((NCellLChoice, nonmodcellsleftchoice))
    NCellRChoice = np.vstack((NCellRChoice, nonmodcellsrightchoice))



LCellLChoice = LCellLChoice[1:, :]
LCellRChoice = LCellRChoice[1:, :]

###########
# get indices for sequence order
newLis = np.argsort(np.argmax((LCellLChoice+LCellRChoice)/2, axis=1))

# resort data so always in sequence order
LCellLChoice = LCellLChoice[newLis, :]
LCellRChoice = LCellRChoice[newLis, :]

RCellLChoice = RCellLChoice[1:, :]
RCellRChoice = RCellRChoice[1:, :]

newRis = np.argsort(np.argmax((RCellRChoice+RCellLChoice)/2, axis=1))

# resort data so always in sequence order
RCellLChoice = RCellLChoice[newRis, :]
RCellRChoice = RCellRChoice[newRis, :]

SCellLChoice = SCellLChoice[1:, :]
SCellRChoice = SCellRChoice[1:, :]

newSis = np.argsort(np.argmax((SCellLChoice+SCellRChoice)/2, axis=1))

# resort data so always in sequence order
SCellLChoice = SCellLChoice[newSis, :]
SCellRChoice = SCellRChoice[newSis, :]

NCellLChoice = NCellLChoice[1:, :]
NCellRChoice = NCellRChoice[1:, :]

newNis = np.argsort(np.argmax((NCellLChoice+NCellRChoice)/2, axis=1))

# resort data so always in sequence order
NCellLChoice = NCellLChoice[newNis, :]
NCellRChoice = NCellRChoice[newNis, :]

print(f'n_neurons: {n_neurons}')
# %matplotlib widget
# plt.plot(alldata[0, 1, :])
plt.close('all')


210
n_neurons: 324


In [5]:
print(f'rightis: \n{rightis}\n\nleftis: \n{leftis} \n\nsplit: \n{splitis} \n\nnon: \n{nonis}')

rightis: 
[  0   1   2   3   4   6   7   8   9  18  20  22  23  27  29  30  38  42
  44  63  66  76  80  81  82  84  88 100 103 104 105 110 114 116 117 118
 119 120 121 126 128 135 140 141 142 143 147 148 153 156 157 167 168 170
 172 173 174 175 183 185 193 194 202 203 205 206 213 214 215 216 225 230
 233 235 240 241 248 252 254 259 263 267 269 272 274 282 286 287 289 293
 294 295 296 298 299 300 301 312 319 320]

leftis: 
[ 10  12  13  15  19  26  34  35  40  41  45  46  47  48  50  53  54  58
  60  68  70  71  73  75  77  79  86  91  94  95  97  99 101 107 109 124
 129 131 136 138 144 146 149 150 151 155 158 166 176 178 179 182 184 187
 189 191 192 198 204 208 209 210 218 219 220 223 226 231 232 234 236 237
 238 239 242 244 250 256 257 258 260 270 271 281 283 284 288 297 302 303
 306 307 308 310 311 314 315 316 322 323] 

split: 
[  5  11  14  17  24  25  28  31  32  33  36  37  39  43  49  51  52  56
  57  59  61  62  64  65  67  69  72  74  78  83  85  87  89  90  92  93
  96  98 1